<a href="https://colab.research.google.com/github/Remil-Maha/ML-Notebooks/blob/main/FakeNewsDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import re
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# LOADING DATA

In [ ]:
df = pd.read_csv('train.csv')


ParserError: Error tokenizing data. C error: EOF inside string starting at row 3993

In [ ]:
df.head()
"""
id: unique id for a news article
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks the article as potentially unreliable
1: unreliable
0: reliable"""

# DATA PREPROCESSING

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop(columns=['author'], inplace=True)

In [ ]:
df.dropna(inplace=True)

**TRUE VS.FAKE DISTRIBUTION**

In [ ]:
category = df['label'].value_counts()
print(category)

In [ ]:
#Plotting the Pie Chart:
plt.figure(figsize=(8,8))
plt.pie(category,labels=category.index , autopct='%1.0f%%')
plt.title('Distribution of FAKE vs.TRUE')
plt.show()

On remarque que les classe sont presque equilibrées

In [ ]:
df.head()

In [ ]:
# remove majuscules  :
df['title'] = df['title'].str.lower()
df['text']= df['text'].str.lower()

In [ ]:
#remove html tags
def remove_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  return soup.get_text()
df['title'] =df['title'].apply(remove_html_tags)
df['text']  =df['text'].apply(remove_html_tags)

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]','',text)
    # The key is to remove the "(Reuters)" string
    # The Model during the training part can memorize it and perfrom great in training and badly when other testing input is given.
    text = re.sub('[()]','',text)
    text = re.sub('\\W',' ',text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
df['text'] = df['text'].apply(wordopt)
df['title'] = df['title'].apply(wordopt)

In [ ]:
nltk.download('stopwords')
# Get English stopwords from NLTK
stop_words = set(stopwords.words('english'))

# Function to remove stop words from text
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['title'] = df['title'].apply(remove_stopwords)
df['text'] = df['text'].apply(remove_stopwords)

**Stemming**

In [ ]:
porter_stemmer = PorterStemmer()
df['title'] = df['title'].apply(lambda x : ' '.join([porter_stemmer.stem(word) for word in x.split()]))
df['text'] = df['text'].apply(lambda x : ' '.join([porter_stemmer.stem(word) for word in x.split()]))

In [ ]:
df.head()

# Model Building

**Text Vectorization**

In [ ]:

vectorizer = TfidfVectorizer()


In [ ]:
df['combined_text'] = df['title'] + " " + df['text']
X = df['combined_text']
Y= df['label']

In [ ]:
X_transformed = vectorizer.fit_transform(X)


In [ ]:
print(X_transformed)
#(document_index, feature_index)    valeur_TFIDF

In [ ]:
lr_model = LogisticRegression(C=0.1 ,solver='liblinear')
lr_model.fit(X_transformed,Y)

# Preparing Test Data:

In [ ]:
df_test_X=  pd.read_csv('test.csv',encoding='latin-1')
df_test_y = pd.read_csv('submit.csv')
df_test = pd.concat([df_test_X, df_test_y], axis=1)


In [ ]:
df_test.head()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.shape

In [ ]:
df_test.drop(columns=['author'], inplace=True)

In [ ]:
df_test.duplicated().sum()

In [ ]:
df_test.dropna(inplace=True)

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.drop(columns=['id'],inplace=True)

In [ ]:
df_test['title'] = df_test['title'].str.lower()
df_test['text']= df_test['text'].str.lower()

In [ ]:
df_test['title'] =df_test['title'].apply(remove_html_tags)
df_test['text']  =df_test['text'].apply(remove_html_tags)

In [ ]:
df_test['text'] = df_test['text'].apply(wordopt)
df_test['title'] = df_test['title'].apply(wordopt)

In [ ]:
df_test['title'] = df_test['title'].apply(remove_stopwords)
df_test['text'] = df_test['text'].apply(remove_stopwords)

In [ ]:
df_test['title'] = df_test['title'].apply(lambda x : ' '.join([porter_stemmer.stem(word) for word in x.split()]))
df_test['text'] = df_test['text'].apply(lambda x : ' '.join([porter_stemmer.stem(word) for word in x.split()]))

In [ ]:
df_test.head()

In [ ]:
df_test['combined_text'] = df_test['title'] + " " + df_test['text']
X_test = df_test['combined_text']

In [ ]:
X_transformed_test = vectorizer.transform(X_test)
lr_pred = lr_model.predict(X_transformed_test)


In [ ]:
y_test =df_test['label']

In [ ]:
lr_accuracy = accuracy_score(y_test, lr_pred)
lr_precision = precision_score(y_test, lr_pred, average='weighted')
lr_recall = recall_score(y_test, lr_pred, average='weighted')
lr_conf_matrix = confusion_matrix(y_test, lr_pred)
print("Accuracy:", lr_accuracy)
print("Precision:", lr_precision)
print("Recall:", lr_recall)
print("Confusion Matrix:\n", lr_conf_matrix)
